<a href="https://colab.research.google.com/github/dudeurv/SAM_MRI/blob/main/choose_best_epoch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def test_per_epoch(model, testloader, ce_loss, dice_loss, multimask_output=True, args=None):
    model.eval()
    loss_per_epoch = []
    num_classes = args.num_classes + 1
    confusion_matrix = np.zeros((num_classes, num_classes), dtype=np.uint32)
    with torch.no_grad():
        for i_batch, sampled_batch in enumerate(testloader):
            image_batch, label_batch, low_res_label_batch = sampled_batch['image'],sampled_batch['label'], sampled_batch['low_res_label']
            image_batch, label_batch, low_res_label_batch = image_batch.to(device, dtype=torch.float32), label_batch.to(device, dtype=torch.long), low_res_label_batch.to(device, dtype=torch.long)

            outputs = model(image_batch, multimask_output, args.img_size)

            logits = outputs['masks']
            prob = F.softmax(logits, dim=1)
            pred_seg = torch.argmax(prob, dim=1)
            confusion_matrix += calculate_confusion_matrix_from_arrays(pred_seg.cpu(), label_batch.cpu(), num_classes)

            loss, loss_ce, loss_dice = calc_loss(outputs, low_res_label_batch, ce_loss, dice_loss)
            loss_per_epoch.append(loss.item())

        confusion_matrix = confusion_matrix[1:, 1:]  # exclude background
        dices_per_class = {'dice_cls:{}'.format(cls + 1): dice
                    for cls, dice in enumerate(calculate_dice(confusion_matrix))}

    return np.mean(loss_per_epoch), dices_per_class

In [ ]:
best_epoch, best_dice = 0.0, 0.0
for epoch in range(args.max_epochs):
    loss_training = training_per_epoch(net, trainloader, optimizer, iter_num, ce_loss, dice_loss, multimask_output=multimask_output, args=args)
    loss_testing, dice = test_per_epoch(net, testloader, ce_loss, dice_loss,multimask_output=True, args=args)
    loss_testing, dice_public = test_per_epoch(net, testloader, ce_loss, dice_loss,multimask_output=True, args=args)

    dice_public_class1 = dice_public['dice_cls:1']
    dice_class1 = dice['dice_cls:1']

    # Average the two class 1 dice scores
    average_dice_class1 = (dice_public_class1 + dice_class1) / 2

    # Combine class 1 average with other class dice scores
    dice_values = np.array([average_dice_class1] + [dice['dice_cls:2']])

    total_dice = np.mean(dice_values)